## Import Libraries 

In [115]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import metrics                           
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score

In [102]:
df_clean = pd.read_csv("Refined_data.csv")
df_clean.head()

/home/deepfintech/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (29) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Unnamed: 0,index,amount_tsh,gps_height,longitude,latitude,wpt_name,basin,subvillage,region,...,management_group,payment,water_quality,quantity,source,source_class,waterpoint_type,status_group,installer_category,funder_categories
0,0,0,6000.0,1390,34.938093,-9.856322,45683,1,13116,3,...,4,2,6,1,8,0,1,functional,15,11
1,1,1,0.0,1399,34.698766,-2.147466,45426,4,17596,9,...,4,0,6,2,5,1,1,functional,15,11
2,2,2,25.0,686,37.460664,-3.821329,17767,5,10096,8,...,4,4,6,1,0,1,2,functional,26,11
3,3,3,0.0,263,38.486161,-11.155298,45537,7,9998,12,...,4,0,6,0,3,0,2,non functional,21,16
4,4,4,0.0,0,31.130847,-1.825359,43411,4,8583,4,...,1,0,6,3,5,1,1,functional,15,11


In [ ]:
scaler = StandardScaler()
scaler

In [109]:
# Selected features from sequential feature selection. 
selected_features=['amount_tsh', 'gps_height', 'longitude', 'latitude', 'wpt_name',
       'basin', 'subvillage', 'region', 'region_code', 'district_code', 'lga',
       'ward', 'population', 'public_meeting', 'scheme_name', 'permit',
       'construction_year', 'extraction_type', 'extraction_type_class',
       'management_group', 'payment', 'water_quality', 'quantity', 'source',
       'source_class', 'waterpoint_type', 'installer_category', 'status_group' ]

df_clean = df_clean[selected_features]

In [122]:
trainX = df_clean[:59400].copy()
trainY=trainX.pop('status_group')

trainX = scaler.fit_transform(trainX)

print ("trainX shape : ", trainX.shape)
print ("trainY shape : ", trainY.shape)

trainX shape :  (59400, 27)
trainY shape :  (59400,)


In [124]:
testX = df_clean[59400:].copy()
testX.pop('status_group')

testX = scaler.fit_transform(testX)

print (testX.shape)

(14850, 27)


### **For the models,** 
* **Random Forest**
* **XGBoost**
* **CatBoost**
* **KNN**

#### **1. Random Forest**

In [125]:
# Random Forest 

def get_acc_rf(x,y):
    rf=RandomForestClassifier(criterion='gini',   #with sqrrfs 0.6948, with out sqrrfs 68.88
                          min_samples_split=5,
                          min_samples_leaf=1,
                          max_features='auto',
                          oob_score=False,
                          random_state=1,
                           n_jobs=-1
                          )
    score=cross_val_score(rf,x,y,cv=5)
    
#   print("score",score)
    print("mean_score",score.mean())
    return score.mean()

get_acc_rf(trainX,trainY)

mean_score 0.8131986531986533


0.8131986531986533

### **2. XGBoost**

In [113]:
# XGBoost 

import xgboost as xgb
from sklearn.model_selection import cross_val_score

def get_acc_xg(trainX,trainY):
    xgboost=xgb.XGBClassifier(objective ="multi:softmax",
                              colsample_bytree = 0.01,
                              learning_rate = 0.001, max_depth = 6,
                              alpha = 0.1,
                              n_estimators = 100 )
    score=cross_val_score(xgboost,trainX,trainY,cv=5)
    

    print("mean_score",score.mean())
    return score.mean()
get_acc_xg(trainX,trainY)

/home/deepfintech/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[09:13:17] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/deepfintech/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[09:13:19] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/deepfintech/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

/home/deepfintech/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[09:13:22] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/deepfintech/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[09:13:24] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
mean_score 0.5929292929292929


0.5929292929292929

### **3. CatBoost**

In [118]:
# CatBoost 
cat=CatBoostClassifier( verbose=False, iterations=1500,
    learning_rate=0.2053434310118264,
    random_strength=8,
    bagging_temperature=0,
    max_bin=20,
    grow_policy="Depthwise",
    min_data_in_leaf=10,
    max_depth=3,
    l2_leaf_reg=9.501510078266123e-06,
    one_hot_max_size=500,
    auto_class_weights="SqrtBalanced")

score=cross_val_score(cat,trainX,trainY,cv=5)
print("mean_score",score.mean())

mean_score 0.7786363636363637


### **4. KNN**

In [116]:
# KNN classification

accuracy_rate={}

for i in range(1,100):
    
    knn = KNeighborsClassifier(n_neighbors=i)
    score=cross_val_score(knn,trainX,trainY,cv=5)
    accuracy_rate[i]=score.mean()
    print(score.mean())

0.5308585858585858
0.5437373737373739
0.546077441077441
0.5477441077441079
0.5454882154882155
0.5502020202020202
0.5477272727272727
0.5483838383838384
0.550976430976431
0.5498316498316498
0.5509932659932659
0.5510101010101011
0.5515993265993265
0.550942760942761
0.5512289562289563
0.5518013468013467
0.5526094276094277
0.5515488215488216
0.5528451178451178
0.5524242424242424
0.5526430976430976
0.5535521885521886
0.5531144781144781
0.552979797979798
0.554090909090909
0.5526094276094276
0.5525420875420874
0.5517845117845118
0.5517845117845118
0.5521885521885521
0.5526599326599326
0.5514646464646465
0.550909090909091
0.5514478114478114
0.5507575757575758
0.5516498316498317
0.5522053872053871
0.5522390572390572
0.5510942760942761
0.5512794612794613
0.5508417508417509
0.5501683501683502
0.5513131313131313
0.5504545454545455
0.5507575757575758
0.5495286195286195
0.5509090909090909
0.5504713804713804
0.5501515151515152
0.5502356902356902
0.5494949494949495
0.5496969696969698
0.5495622895622896

***
**Highest acccuracy is for Random Forest Classifier**

**Created the model with Random Forest**

In [126]:
rf=RandomForestClassifier(criterion='gini',   
                          min_samples_split=5,
                          min_samples_leaf=1,
                        max_features='auto',
                           oob_score=False,
                           random_state=1,
                            n_jobs=-1)
rf.fit(trainX,trainY)

RandomForestClassifier(min_samples_split=5, n_jobs=-1, random_state=1)

In [127]:
predictions=rf.predict(testX)

df_output=pd.read_csv("data/Submission format.csv",index_col="id")
df_output['status_group']=predictions
df_output.to_csv("submission_file.csv")
    
print (len(predictions))

14850
